# Model Creation

In this jupyter notebook we shall look at taking the preprocessed data  generated by preprocessing_part_2.ipynb and creating machine learning model from it 
that reads each review and tries to predict what its average score is. Thus we are building a text classifier

In [128]:
#start with the relevant imports

#use to visualise the data 
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

#used to build the model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.optimizers import RMSprop
from keras.losses import SparseCategoricalCrossentropy

Step 1: load and inspect the csv with pandas

In [129]:
#first load the data with pandas
df=pd.read_csv("./data/data_ready_for_model.csv")


In [130]:
df.head(10)

,Unnamed: 0,Comments,Average Score
0,0,moved uk end august got virgin media broadband...,1.0
1,1,truly attrocious service terms broadband custo...,1.0
2,2,hard cancel contract. phone 2 hours t o spend ...,2.0
3,3,pay 350mbps package managed 250mbps upload 34 ...,2.0
4,4,worst customer service: -the bots ask irreleva...,2.0
5,5,informed given upgrade difference speeds reboo...,3.0
6,6,wish zero star virgin media unfortunately lowe...,1.0
7,7,sold package 1gb speed 2 tivo box 6 ask act li...,2.0
8,8,simply don't sold. example switching bt 67mb p...,3.0
9,9,virgin worse broadband company. ive trying boo...,1.0


In [131]:
df.drop("Unnamed: 0", axis=1, inplace=True) #unneeded column, resulted when csv was created from dataframe

The last step before splitting our data into train test split sets is to tokenize the words.

In [132]:
#max words to be used.
max_words=5000 
#max no of words per complaint:
max_sequence=250
#fixed
embedding_dim=250

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df["Comments"].values)
word_index=tokenizer.word_index

Truncate and pad the input sequences so that they are all in the same length for modeling.

In [133]:
print(f"found {len(word_index)} unique tokens")

found 12592 unique tokens


In [134]:
X = tokenizer.texts_to_sequences(df['Comments'].values)
X = tf.keras.utils.pad_sequences(X, maxlen=max_sequence)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4342, 250)


Step 2 prepare the data into train val test sets (code is borrowed from my Wine reviews classification Neural Network). We want our target ot be our "average score" and our features to be the "comments". We have quite the imbalanced dataset,  because we have more average scores with a score of 1 and two than any other score. Because we are implementing a classification model, this could be especially problematic.

To overcome this data we will _stratify_ the data. This is to ensure that relative class frequencies is approximately preserved in each train and validation fold.

In [135]:
y=df["Average Score"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.340, random_state=0, stratify=y)
#60 training, 20 validation, 20 testing
X_val, X_test, y_val, y_test =train_test_split(X_temp, y_temp, test_size = 0.5, random_state=0, stratify=y_temp)

# Training an LSTM model

the time for creating a neural network has finally arrived! First, let's encode our comments using a text vectorizor model:

let's check our vocabulary. These are just some of the words that have been encoded into vectors: (UNK) represents any unknown tokens

In [136]:
model = Sequential([
        Embedding(max_words, embedding_dim, input_length=X.shape[1]),#mask=0 so we can handle inputs of variable lengths
        #now we have a vector of numbers a nn can comprehend
        tf.keras.layers.SpatialDropout1D(0.2),
        tf.keras.layers.LSTM(32),
        Dense(32, activation="relu"),
        Dropout(0.4),
        Dense(5, activation="softmax")
])

In [137]:
callback = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='saved_model', monitor='val_loss', save_best_only=True)]



In [138]:
model.compile(RMSprop(learning_rate=0.001), 
             loss = SparseCategoricalCrossentropy(), #categorical cross entropy as multi classification problem
                metrics=["sparse_categorical_accuracy"])

In [139]:
model.evaluate(X_train, y_train) #evaluate performance of model without training it first
#accuracy is around 0.36.7

90/90 [==============================] - 4s 35ms/step - loss: 1.6114 - sparse_categorical_accuracy: 0.1194


[1.611374020576477, 0.11937172710895538]

In [140]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=callback)

Epoch 1/50
90/90 [==============================] - ETA: 0s - loss: 1.0164 - sparse_categorical_accuracy: 0.6230

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


90/90 [==============================] - 17s 161ms/step - loss: 1.0164 - sparse_categorical_accuracy: 0.6230 - val_loss: 0.8620 - val_sparse_categorical_accuracy: 0.6491
Epoch 2/50
90/90 [==============================] - 12s 131ms/step - loss: 0.8811 - sparse_categorical_accuracy: 0.6360 - val_loss: 0.8640 - val_sparse_categorical_accuracy: 0.6491
Epoch 3/50
90/90 [==============================] - 12s 129ms/step - loss: 0.7998 - sparse_categorical_accuracy: 0.6565 - val_loss: 0.9013 - val_sparse_categorical_accuracy: 0.6531
Epoch 4/50
90/90 [==============================] - 12s 128ms/step - loss: 0.6916 - sparse_categorical_accuracy: 0.7082 - val_loss: 1.0129 - val_sparse_categorical_accuracy: 0.6206
Epoch 5/50
90/90 [==============================] - 12s 129ms/step - loss: 0.5610 - sparse_categorical_accuracy: 0.7742 - val_loss: 1.1174 - val_sparse_categorical_accuracy: 0.5583
Epoch 6/50
90/90 [==============================] - 11s 126ms/step - loss: 0.4535 - sparse_categorical_acc

In [141]:
model.save("saved_model/model1")

INFO:tensorflow:Assets written to: saved_model/model1\assets


INFO:tensorflow:Assets written to: saved_model/model1\assets


In [142]:
df["Average Score"].value_counts()

2.0    2819
1.0    1227
3.0     275
4.0      21
Name: Average Score, dtype: int64

Model has trained but has a val accuracy of only 0.6531. We can see that it is clearly overfitting. We have quite an imbalanced dataset, that we need to account for. Lots of 2.0 star reviews and very few  3 and 4 start reviews.


To increase the accuracy, it might be worth merging the two smallest classes together with a combined rating of 3.5:

In addition, to tackle the large number of 2.0 and 1.0 start reviews, we will be using Jaccard's similarity to look for reviews that are similar to each other enough to be counted as duplicates and then remove them. jaccard's similarity is a mathematical function that just does that: measures how similar two sets are to each other.

First, let's address the two smaller classes and merge them together.


In [143]:
#merge smallest classes in the df  #softmax will now be 3 classes
condition = df['Average Score']== 4
df.loc[condition, 'Average Score'] = 3.5
condition = df['Average Score']== 3
df.loc[condition, 'Average Score'] = 3.5

In [144]:
df["Average Score"].value_counts()

2.0    2819
1.0    1227
3.5     296
Name: Average Score, dtype: int64

Now let's deal with the comments. First tokenize the comments column and define the jaccard function. 

In [145]:
tokenize = lambda doc: doc.lower().split(" ")
tokenized_documents = [tokenize(d) for d in df["Comments"]] # tokenized docs

In [146]:
print(tokenized_documents[6])

['wish', 'zero', 'star', 'virgin', 'media', 'unfortunately', 'lowest', 'star', '1.', 'intend', 'joining', 'network', "don't", 'want', 'bills', 'increasing', 'informed', "don't", 'mistake', 'joing', 'virgin', 'media.', 'set', 'liars', 'lied', '2019', 'join', 'network', 'pay', '33', 'pounds', '18', 'month', 'contract.', '4', 'month', 'virgin', 'media', 'increase', 'price', '59', 'pounds', 'month', "wasn't", 'initially', 'told', 'me.', 'realised', "they've", 'increased', 'monthly', 'end', 'contract', 'ended', 'contract', '102.27', 'insisted', 'pay', 'began', 'send', 'debt', 'collector', 'disturb', 'payment', 'spoilt', 'credit', 'rating', 'reported', 'case', 'credit', 'authority', 'credit', 'affected', 'them.', 'love', 'credit', 'report', "don't", 'want', 'paying', 'extra', 'bills', 'comparison', 'initially', 'agree', "don't", 'join', 'virgin', 'media.', 'lastly', 'saw', "they've", 'gone', 'bit', 'spoil', 'credit', 'report', 'contacted', 'voice', 'complain', 'customer', 'service', 'listene

In [147]:
def jaccard_similarity(query: set, document: set) -> float:
    
    """"Returns the Jaccard similarity between a query and a specified document"""

    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))

Let's apply this measure to the comments column! As the most common score is the one with an average of two, we will try and eliminate reviews that have comments that have two star ratings, or at least look like they could have two star ratings.

To do this, I will (arbitrarily) pick the first comment that has a 2 star review and compare the rest of the comments in the dataframe to this one using the Jaccardian metric.

In [163]:
values=[]
def jaccard_similarity(query: set) -> float:
    
    """"Returns the Jaccard similarity between a query and a specified document"""

    intersection = set(query).intersection(set(df["Comments"][0]))
    union = set(query).union(set(df["Comments"][0]))
    values.append(len(intersection)/len(union))

In [164]:
df["Comments"].apply(jaccard_similarity)

0       None
1       None
2       None
3       None
4       None
        ... 
4337    None
4338    None
4339    None
4340    None
4341    None
Name: Comments, Length: 4342, dtype: object

In [165]:
values

[1.0,
 0.7027027027027027,
 0.6666666666666666,
 0.6923076923076923,
 0.6923076923076923,
 0.5277777777777778,
 0.825,
 0.6486486486486487,
 0.7142857142857143,
 0.6756756756756757,
 0.7631578947368421,
 0.7297297297297297,
 0.6756756756756757,
 0.7631578947368421,
 0.5,
 0.4166666666666667,
 0.7631578947368421,
 0.6486486486486487,
 0.7,
 0.7894736842105263,
 0.6,
 0.8378378378378378,
 0.7692307692307693,
 0.7368421052631579,
 0.7567567567567568,
 0.6923076923076923,
 0.7222222222222222,
 0.775,
 0.6410256410256411,
 0.5675675675675675,
 0.6578947368421053,
 0.6486486486486487,
 0.6578947368421053,
 0.5555555555555556,
 0.825,
 0.6944444444444444,
 0.3888888888888889,
 0.6585365853658537,
 0.6666666666666666,
 0.5555555555555556,
 0.7027027027027027,
 0.7222222222222222,
 0.717948717948718,
 0.631578947368421,
 0.6756756756756757,
 0.5833333333333334,
 0.6578947368421053,
 0.5405405405405406,
 0.75,
 0.6944444444444444,
 0.7380952380952381,
 0.7631578947368421,
 0.6944444444444444,
 0

Crea

In [168]:
df["Jaccard Similarity"]=values

In [ ]:
df.head()

,Comments,Average Score,Jaccard Similarity
0,moved uk end august got virgin media broadband...,1.0,1.000000
1,truly attrocious service terms broadband custo...,1.0,0.702703
2,hard cancel contract. phone 2 hours t o spend ...,2.0,0.666667
3,pay 350mbps package managed 250mbps upload 34 ...,2.0,0.692308
4,worst customer service: -the bots ask irreleva...,2.0,0.692308


In [ ]:
df.head()

,Comments,Average Score,jaccard similarity
0,moved uk end august got virgin media broadband...,1.0,0.008274
1,truly attrocious service terms broadband custo...,1.0,0.005988
2,hard cancel contract. phone 2 hours t o spend ...,2.0,0.005985
3,pay 350mbps package managed 250mbps upload 34 ...,2.0,0.006214
4,worst customer service: -the bots ask irreleva...,2.0,0.006214


In [ ]:
y=df["Average Score"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.340, random_state=0, stratify=y)
#60 training, 20 validation, 20 testing
X_val, X_test, y_val, y_test =train_test_split(X_temp, y_temp, test_size = 0.5, random_state=0, stratify=y_temp)

In [ ]:
model2 = Sequential([
        Embedding(max_words, embedding_dim, input_length=X.shape[1]),#mask=0 so we can handle inputs of variable lengths
        #now we have a vector of numbers a nn can comprehend
        tf.keras.layers.SpatialDropout1D(0.2),
        tf.keras.layers.LSTM(32),
        Dense(32, activation="relu"),
        Dropout(0.4),
        Dense(4, activation="softmax")
])

In [ ]:
model2.compile(RMSprop(learning_rate=0.001), 
             loss = SparseCategoricalCrossentropy(), #categorical cross entropy as multi classification problem
                metrics=["sparse_categorical_accuracy"])

In [ ]:
model2.evaluate(X_train, y_train) #evaluate performance of model without training it firs

90/90 [==============================] - 5s 43ms/step - loss: 1.3882 - sparse_categorical_accuracy: 0.1490


[1.3882297277450562, 0.1490401327610016]